In [1]:
import chromadb
import torch
import re
import os


from chromadb import PersistentClient



from transformers import (
    AutoModel, AutoTokenizer,AutoModelForCausalLM,TrainingArguments, DataCollatorForLanguageModeling, Trainer
)

from week5.tools.get_embedding import get_embedding

from datasets import load_dataset

from peft import PeftModel

In [2]:
qwen_model_path = '/home/wby/projects/model/Qwen-7B-Chat'
db_path = '/home/wby/projects/week5/chroma_db/chroma_recipe_db'
model_path = '/home/wby/projects/model/all-MiniLM-L6-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
#连接数据库
chroma_client = PersistentClient(
    path=db_path,
    settings=chromadb.config.Settings(
        anonymized_telemetry=False,
        allow_reset=False
    )
)

In [4]:
#查看我的数据库里有多少集合
collections = chroma_client.list_collections()
print(f"数据库里共有{len(collections)}个集合")
for coll in collections:
    print(f"集合名称{coll.name}")

数据库里共有1个集合
集合名称recipe_50w_384d_ver2


In [5]:
#获取集合，连接模型是使用rag的基础
collection = chroma_client.get_collection(name='recipe_50w_384d_ver2')
model = AutoModel.from_pretrained(model_path).to(device) #注意！这是小模型 allMiniLM-V6
tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=True) #注意！allMiniLM-V6的分词器

In [6]:
def parse_conditions(query):
    """
    从用户查询中提取条件，比如calories < 200  time > 10 转为Chroma支持的过滤格式
    :param query: 询问的参数
    :return: Chroma过滤字典（如{"Calories": {"$lt": 200}, "TotalTime": {"$gt": 10}}）
    """
    field_mapping = {
        # 键：用户可能输入的关键词（小写）；值：元数据中对应的字段名
        "calorie": "Calories",
        "calories": "Calories",
        "kcal": "Calories",
        "time": "TotalTime",
        "totaltime": "TotalTime",
        "duration": "TotalTime",
        "serving": "RecipeServings",
        "servings": "RecipeServings",
        "portion": "RecipeServings",
        "recipe servings": "RecipeServings",
        "RecipeServings": "RecipeServings",
    }

    # 2. 定义运算符映射：用户输入的符号 → Chroma支持的运算符
    operator_mapping = {
        "<": "$lt",
        "<=": "$lte",
        ">": "$gt",
        ">=": "$gte",
        "=": "$eq",
        "==": "$eq",
        'less than':"$lt",
        'greater than':"$gt",
        'less than or equal to':"$lte",
        'greater than or equal to':"$gte",
        'equal to':"$eq",
    }

    # 3. 用正则表达式提取所有条件（支持多条件，用and连接）
    # 匹配格式：[关键词] [运算符] [数字]（如 "calories < 200"、"time >= 10"）
    pattern = r"(\w+\s*\w*)\s*([<>=]{1,2})\s*(\d+\.?\d*)"
    matches = re.findall(pattern, query.lower())  # 转小写，忽略大小写

    #4. 处理提取到的条件，转换为Chroma格式
    conditions = []
    for match in matches:
        # match = (用户输入关键词, 运算符, 数值)
        user_filed, user_op, value = match

        #首先去除关键词中的空格
        # 去除关键词中的空格（如"total time" → "totaltime"）
        user_filed_clean= user_filed.replace(" ","")
        meta_field = None
        for key in field_mapping:
            if key in user_filed_clean:
                meta_field = field_mapping[key]
                break

        if not meta_field:
            continue #不认识的字段直接跳过

         # 4.2 转换运算符（如"<" → "$lt"）
        chroma_op = operator_mapping.get(user_op,None)
        if not chroma_op:
            continue # 不支持的运算符， 跳过

        try:
            value = float(value)
        except:
            continue  #数值无效，跳过

        conditions.append({meta_field: {chroma_op: value}})

    if len(conditions) == 0:
         return None # 没有条件， 后记组合时chroma会过滤的
    elif len(conditions) == 1:
        return conditions[0] # 单个条件直接返回
    else:
        return {'$and':conditions}



In [7]:
query1 = "Find Low-Fat Dessert with calories < 200"
print("测试1结果：", parse_conditions(query1))
# 预期输出：{"Calories": {"$lt": 200}}

# 测试用例2：多个条件（耗时>10且<30，份量=2）
query2 = "Recipe with total time >10 and total time <30, servings = 2"
print("测试2结果：", parse_conditions(query2))
# 预期输出：{"$and": [{"TotalTime": {"$gt": 10}}, {"TotalTime": {"$lt": 30}}, {"RecipeServings": {"$eq": 2}}]}

 # 测试用例3：带小数的条件（热量>=150.5）
query3 = "kcal >= 150.5"
print("测试3结果：", parse_conditions(query3))
# 预期输出：{"Calories": {"$gte": 150.5}}

# 测试用例4：无有效条件
query4 = "Find Banana Flambe recipe"
print("测试4结果：", parse_conditions(query4))

测试1结果： {'Calories': {'$lt': 200.0}}
测试2结果： {'$and': [{'TotalTime': {'$gt': 10.0}}, {'TotalTime': {'$lt': 30.0}}, {'RecipeServings': {'$eq': 2.0}}]}
测试3结果： {'Calories': {'$gte': 150.5}}
测试4结果： None


In [8]:
"""query_embedding = get_embedding('quick banana flambe',tokenizer, model,device)"""

def rag_retrieve(query,top_k=3):
    """
    基础的rag函数，根据查询返回最最相似的食谱
    :param query: 用户查询的问题
    :param top_k: 返回前k个最相近的结果
    :return 返回包括食谱信息的列表|
    """
    query_emb = get_embedding(query,tokenizer, model,device)

    #添加条件
    filter_conditions = parse_conditions(query)

    resluts = collection.query(
        query_embeddings = [query_emb],
        n_results=top_k,
        include=['metadatas','documents','distances'],
        where= filter_conditions
    )
    """print(f"这是一个测试ids{resluts['ids']}")
    print(f"这是一个测试ids{resluts['distances']}")
    print(f"这是一个测试ids{resluts['metadatas']}")
    print(f"这是一个测试documents{resluts['documents']}")"""
    formatted_resluts = []
    for i,(doc_id,distance,meta, doc_text) in enumerate(zip(
        resluts['ids'][0], resluts['distances'][0],
        resluts['metadatas'][0], resluts['documents'][0]
    )):
        formatted_resluts.append({
            'doc_id': doc_id,
            'distance': distance,
            'meta': meta,
            'text': doc_text
        })
    return formatted_resluts



In [9]:
query = "calories < 250 and vegetable based"
lists = rag_retrieve(query,5)
print(lists)

[{'doc_id': '278655_1922', 'distance': 0.34781521558761597, 'meta': {'Calories': 32.2, 'Name': "My Mother's Version: Weight Watcher's 0 Points Vegetable Soup", 'RecipeCategory': 'Vegetable', 'RecipeId': 278655, 'RecipeServings': 6.0, 'TotalTime': 65.0}, 'text': "RecipeId:278655\nName:My Mother's Version: Weight Watcher's 0 Points Vegetable Soup\nTotalTime:65.0\nRecipeCategory:Vegetable\nCalories:32.2\nRecipeServings:6.0\nIngredients and Quantities:\nonion:1/2\ncarrot:1\ncelery ribs:1 1/2\nnonfat beef broth:3\ndiced tomatoes:1\ncabbage:2 1/2\nbeef bouillon cubes:2\ngarlic cloves:2\nbasil:1\nzucchini:1/4\nCooking steps:\n1. Spray large saucepan with cooking spray.\n2. Saute' onion, carrots and celery until tender, approximately 5 minutes.\n3. Combine beef broth, tomatoes, cabbage, bouillon cubes, garlic, Cajun spice, and basil in the saucepan.\n4. Bring to a boil; reduce heat; simmer approximately 30 minutes.\n5. Add zucchini, continue simmering for another 10 minutes.\n6. Serve hot.\n7.

In [10]:

def load_qwen_tokenizer(qwen_model_path):
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            qwen_model_path,
            trust_remote_code = True
        )
        qwen_official_chat_template = """
        {% for message in messages %}
            {% if message['role'] == 'user' %}
                <|im_start|>user
                {{ message['content'] }}
                <|im_end|>
            {% elif message['role'] == 'assistant' %}
                <|im_start|>assistant
                {{ message['content'] }}
                <|im_end|>
            {% endif %}
        {% endfor %}
        {% if add_generation_prompt %}
            <|im_start|>assistant
        {% endif %}
        """
        tokenizer.chat_template = qwen_official_chat_template  # 把模板赋值给分词器
        tokenizer.pad_token = tokenizer.eos_token  # 用结束符作为填充符（官方推荐）
        print('Qwen 分词器加载成功！')
        return tokenizer
    except Exception as e:
        print(e)
        return None

qwen_tokenizer = load_qwen_tokenizer(qwen_model_path)

Qwen 分词器加载成功！


In [11]:

def load_qwen_model(qwen_model_path):
    try:
        print("开始加载模型，模型较大，耐心等待，祈祷没人占用GPU")
        model = AutoModelForCausalLM.from_pretrained(
            qwen_model_path,
            device_map = 'auto',
            trust_remote_code = True,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
        ).eval()
        print('Qwen-7B-Chat 模型加载成功')
        return model

    except Exception as e:
        error_msg = str(e)
        if "out of memory" in error_msg.lower():
            print(f"❌ 显存不足！解决方案：1. 改用 CPU 加载（把 device_map 设为 'cpu'）；2. 用 Int4 量化模型（需装 auto-gptq）")
        elif "No such file or directory" in error_msg:
            print(f"❌ 模型路径错误！请确认 {model_path} 下有 config.json、pytorch_model-*.bin 等文件")
        else:
            print(f"❌ 模型加载失败：{error_msg}")
        return None

In [12]:
if qwen_tokenizer:
    base_model = load_qwen_model(qwen_model_path)
    qwen_model = PeftModel.from_pretrained(base_model, "/home/wby/projects/week5/data/qwen_food_lora3/final_lora")
    #qwen_model = load_qwen_model(qwen_model_path)
else:
    print('分词器未加载，无法加载模型')

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


开始加载模型，模型较大，耐心等待，祈祷没人占用GPU


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Qwen-7B-Chat 模型加载成功


In [13]:
# 尝试打印精确版本（部分早期版本可能没有，不影响使用）
try:
    print("精确版本：", qwen_model.__version__)
except AttributeError:
    print("精确版本未标注，按现有配置正常使用即可")

精确版本未标注，按现有配置正常使用即可


In [32]:
prompt_template = """
###任务：严格根据用户JSON中的Name字段，返回对应食谱的完整信息（缺一不可）
1. 只读取用户JSON里的"Name"值，忽略其他字段；
2. 输出必须包含所有字段：Name（和输入Name完全一致）、RecipeCategory、RecipeServings、TotalTime、Ingredients、Step；
3. 仅输出JSON，无任何多余内容，字段值用英文。

###示例：
用户查询：{"Name":"chicken salad","Calories":250}
输出：
{
    "Name": "chicken salad",
    "RecipeCategory": "Low Calorie",
    "RecipeServings": "4",
    "TotalTime": "30 min",
    "ingredients": "chicken breast:1 lb\\nmayo:1/2 cup\\ndill pickles:6 slices\\nred onion:1/4 cup",
    "cookingSteps": "1. Cook chicken and shred.\\n2. Mix with mayo, pickles and onion.\\n3. Chill before serving."
}

###用户查询：
{query}

###输出（必须包含Step字段）：
"""

# 若检索不出用户提到的菜名，则让模型自己检索
def is_food_teach(query, qwen_model, qwen_tokenizer):
    """
    基于 QWen-7B 模型，用QWEN-7B自己来给RAG检索之外的答案
    :param query: 用户查询（字符串）
    :param qwen_model: 加载好的 QWen-7B 模型
    :param qwen_tokenizer: 加载好的 QWen 分词器
    :return:
    """
    # 1. 构造提示词模板（替换 {{query}} 为实际用户查询）

    # 填充用户查询到提示词中
    #prompt = prompt_template.format(query=query)
    final_prompt = prompt_template.replace('{query}', query)

    # 2. 用 QWen 分词器处理提示词（适配 QWen 聊天格式）
    inputs = qwen_tokenizer.apply_chat_template(
        [
            {'role':"system", "content": '你是一个专业的美食顾问，你需要根据用户询问，判定他是需要你推荐美食还是提供这道菜的食谱'},
            {"role": "user", "content": final_prompt}
        ],  # 按 QWen 要求的聊天格式组织
        add_generation_prompt=True,  # 自动添加模型回复的前缀
        return_tensors="pt"  # 返回 PyTorch 张量
    ).to(qwen_model.device)  # 移到模型所在设备（GPU/CPU）

    # 3. 模型生成结果（控制输出长度，避免冗余）
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=4000,  # 食物列表长度有限，50个token足够
            temperature=0.1,  # 降低随机性，确保输出格式稳定
            top_p=0.9,
            do_sample=False,  # 确定性生成，避免重复结果
            eos_token_id=qwen_tokenizer.eod_id  # 遇到结束符停止生成
        )

    # 4. 解码输出并提取食物列表
    # 提取模型生成的部分（排除输入提示词）
    generated_ids = outputs[:, inputs.shape[1]:]
    raw_output = qwen_tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

    return raw_output


In [33]:
query = """{
    "Name":'Tomato Soup',
    'Calories':604.3,
    "Total Time":30,
    "Reason": 'vegetable',
    'taste' : 'sour'
}"""
reslut = is_food_teach(query, qwen_model, qwen_tokenizer)
print(reslut)

{
            "Name": "Tomato Soup",
            "RecipeCategory": "Soup",
            "RecipeServings": "6",
            "TotalTime": "30 min",
            "reason": "vegetable",
            "taste": "sour",
            "ingredients": "tomatoes:6 medium\nonion:1 medium\ngarlic:3 cloves\ncarrots:2 medium\ncelery:2 stalks\nlow-fat milk:2 cups\nwater:4 cups\nsalt:1 tsp\npepper:1/2 tsp",
            "cookingSteps": "1. In a large pot, sauté onion and garlic until softened.\n2. Add carrots and celery, cook for 5 more minutes.\n3. Add tomatoes, milk, water, salt, and pepper.\n4. Bring to a boil, then reduce heat and simmer for 20 minutes.\n5. Use an immersion blender or transfer soup to a blender and blend until smooth.\n6. Serve hot."
        }


In [13]:
#构建根据食材推荐的提示词
def build_recipe_prompt(query, retrieved_results, prompt_template):
    """"

    """
    retrieved_str = ""
    for i,result in enumerate(retrieved_results,1):
        doc_id = result['doc_id']
        distance = round(result['distance'],1)
        recipe_name = result['meta']['Name']
        total_time = round(result['meta']['TotalTime'],1)
        doc_text = result['text']
        calories = round(result['meta']['Calories'],1)
        category = result['meta']['RecipeCategory']

        #拼接单条数据
        retrieved_str += f"""
        第{i}条数据结果：
        - 文档ID: {doc_id}
        - 相似度: {distance}
        -  名称: {recipe_name}
        -  种类: {category}
        - 总时间: {total_time} min
        -  热量: {calories} 卡
        -食谱核心: {doc_text}
        """

    final_template = prompt_template.replace("{{query}}", query).replace("{{retrieved_results}}", retrieved_str)
    return final_template


In [14]:
# 1. 核心搭配提取（不变，处理带动作的query）
def extract_core_pair(query):
    food_actions = ["eat", "cook","cooking", "make", "bake", "fry", "prepare", "taste"]
    # 1. 先清理特殊符号（问号、小于号等）
    query_clean = re.sub(r'[?.,!;<>≤≥]', '', query).strip().lower()
    # 2. 过滤“时间/热量”等非食物条件词（关键新增）
    condition_words = ["time", "min", "minute", "hour"]
    for word in condition_words:
        query_clean = re.sub(rf'\b{word}\b\s*[0-9]*', '', query_clean).strip()

    # 3. 提取“动作+食物”核心搭配
    for action in food_actions:
        pattern = rf'\b{action}\b\s*(.*)'
        match = re.search(pattern, query_clean)
        if match:
            core_object = match.group(1).strip()
            stop_words = ["some", "the", "a", "an", "my", "your"]
            for stop in stop_words:
                core_object = re.sub(rf'\b{stop}\b', '', core_object).strip()
            if core_object:
                core_pair = f"{action} {core_object}"
                print(f"🎯 提取核心搭配：[{query}] → [{core_pair}]")
                return core_pair
    return query_clean

# 2. 单字词补全（不变）
def complete_single_word_query(query):
    if len(query.strip().split()) == 1:
        completed_query = f"eat {query.strip()}"
        return completed_query
    if len(query.strip().split()) == 2:
        completed_query = f"how to make {query.strip()}"
        return completed_query
    return query

# 3. 最终判断函数（新增“纯食物名词”规则）
def is_food_query_final_solution(query, model, tokenizer):
    """
    判定一段对话是否是和讨论美食相关的话题
    """
    core_query = extract_core_pair(query)
    completed_query = complete_single_word_query(core_query)

    # 关键新增：规则同时包含“动作+食材”和“纯食物名词组合”
    judge_prompt = f"""
    用户类型： 那些讨论美食相关话题的用户
    规则：1. 满足以下任一条件→YES，否则NO，仅输出YES/NO。
         2. 判定用户话题如果是与食物相关的->YES, 否则NO, 仅输出YES/NO
    1. 含eat/cook/cooking/make/bake/fry/about+食材（如make banana dessert→YES）；
    2. 纯食物名词组合.（列如milk）
    示例1：eat apple→YES；示例2：banana dessert→YES；示例3：use phone→NO。
    现在判断：{completed_query}→
    """


    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [
                {'role':"system", "content": '你是一个专业的吃货，只需判读这段对话是不是在讨论与美食相关的话题'},
                {"role": "user", "content": judge_prompt},
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        outputs = model.generate(
            inputs,
            max_new_tokens=2,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones_like(inputs)
        )

    #食物白名单
    witeName = {
        'milk','yu xiang shredded pork', 'high calorie food', 'low calorie food', 'family food','sichuan boiled fish','flour'
    }
    has_wite = any(s in completed_query for s in witeName)
    #如果有白名单食物，则返回yes就行，其它的就按照模型判断
    if has_wite:
        print(f"🔍 是否是食物核心搭配判断：[{completed_query}] → 输出：[YES]")
        return 'YES'

    raw_output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).strip()
    clean_output = re.sub(r'[。.\s]', '', raw_output).upper()
    print(f"🔍 是否是食物核心搭配判断：[{completed_query}] → 输出：[{raw_output}]")
    return clean_output

In [15]:
# 测试
query1 = "protein-rich"
result1 = is_food_query_final_solution(query1, qwen_model, qwen_tokenizer)
print(f"测试1（{query1}）返回：{result1}\n")  # 预期：YES

query2 = "egg fried rice"
result2 = is_food_query_final_solution(query2, qwen_model, qwen_tokenizer)
print(f"测试2（{query2}）返回：{result2}\n")  # 预期：NO

query3 = "kung pao chicken"
result3 = is_food_query_final_solution(query3, qwen_model, qwen_tokenizer)
print(f"测试3（{query3}）返回：{result3}\n")  # 预期：YES

query4 = "calorie"
result4 = is_food_query_final_solution(query4, qwen_model, qwen_tokenizer)
print(f"测试4（{query4}）返回：{result4}\n")  # 预期：YES

query5 = "yu xiang shredded pork"
result5 = is_food_query_final_solution(query5, qwen_model, qwen_tokenizer)
print(f"测试5（{query5}）返回：{result5}\n")  # 预期：NO

/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


🔍 是否是食物核心搭配判断：[eat protein-rich] → 输出：[NO]
测试1（protein-rich）返回：NO

🔍 是否是食物核心搭配判断：[egg fried rice] → 输出：[YES]
测试2（egg fried rice）返回：YES

🔍 是否是食物核心搭配判断：[kung pao chicken] → 输出：[NO]
测试3（kung pao chicken）返回：NO

🔍 是否是食物核心搭配判断：[eat calorie] → 输出：[NO]
测试4（calorie）返回：NO

🔍 是否是食物核心搭配判断：[yu xiang shredded pork] → 输出：[YES]
测试5（yu xiang shredded pork）返回：YES



In [16]:
# 验证适配器是否加载成功
def check_lora_loaded(model):
    # 检查模型是否包含LoRA适配器参数
    has_lora = any("lora" in name for name, _ in model.named_parameters())
    print(f"LoRA适配器是否加载：{'✅ 已加载' if has_lora else '❌ 未加载'}")
    # 检查模型精度
    dtype = next(model.parameters()).dtype
    print(f"模型精度：{dtype}（应为torch.float16）")

# 在测试代码中加入验证
check_lora_loaded(qwen_model)  # 替换成你的模型变量名

LoRA适配器是否加载：✅ 已加载
模型精度：torch.bfloat16（应为torch.float16）


In [18]:
# 提取一句话中的所有食物相关的单词
def is_food_list(query, qwen_model, qwen_tokenizer):
    """
    基于 QWen-7B 模型，提取查询中的食物名词列表
    :param query: 用户查询（字符串）
    :param qwen_model: 加载好的 QWen-7B 模型
    :param qwen_tokenizer: 加载好的 QWen 分词器
    :return: 食物名词列表（如 ['apple pie']）
    """
    core_query = extract_core_pair(query)
    # 1. 构造提示词模板（替换 {{query}} 为实际用户查询）
    prompt_template = f"""###任务：基于用户的询问，提取话语中的与食物相关的名词（只能提取食物名词）
    用户查询（query）：{core_query}

    ### 示例：
    1. query = 'i want to cook french fries' 输出：['french fries']
    2. query = 'i want to cook apple pie and orange' 输出：['apple pie' , 'orange']
    3. query = 'i want to make apple pie and Trump' 输出：['apple pie']

    ### 注意事项:
    1. 你只能提取与食物有关的单词。
    2. 不能提取与食物无关的单词（例如人名、地名）

    ###输出格式
    请按照以下格式输出：
    ['food1','food2','food3']"""

    # 填充用户查询到提示词中
    #prompt = prompt_template.format(query=query)

    # 2. 用 QWen 分词器处理提示词（适配 QWen 聊天格式）
    inputs = qwen_tokenizer.apply_chat_template(
        [
            {'role':"system", "content": '你是一个专业的吃货，需要从用户的一段话里提取出所有与食物有关的单词'},
            {"role": "user", "content": prompt_template}
        ],  # 按 QWen 要求的聊天格式组织
        add_generation_prompt=True,  # 自动添加模型回复的前缀
        return_tensors="pt"  # 返回 PyTorch 张量
    ).to(qwen_model.device)  # 移到模型所在设备（GPU/CPU）

    # 3. 模型生成结果（控制输出长度，避免冗余）
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=50,  # 食物列表长度有限，50个token足够
            temperature=0.1,  # 降低随机性，确保输出格式稳定
            top_p=0.9,
            do_sample=False,  # 确定性生成，避免重复结果
            eos_token_id=qwen_tokenizer.eod_id  # 遇到结束符停止生成
        )

    # 4. 解码输出并提取食物列表
    # 提取模型生成的部分（排除输入提示词）
    generated_ids = outputs[:, inputs.shape[1]:]
    raw_output = qwen_tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

    # 用正则表达式提取 ['food1','food2'] 格式的内容
    # 匹配单引号/双引号包裹的字符串，支持中英文食物名
    food_pattern = r"['\"]([^'\"]+)['\"]"
    food_list = re.findall(food_pattern, raw_output)

    # 5. 过滤空值（避免模型输出格式异常导致的空元素）
    food_list = [food.strip() for food in food_list if food.strip()]
    print(f"提取出的食物关键词{food_list}")

    return food_list


In [19]:
# 2. 测试查询
query1 = 'recommend protein-rich low-carb foods'
result4 = is_food_list(query1, qwen_model, qwen_tokenizer)
print(f"查询：{query1}")
print(f"提取的食物列表：{result4}")  # 预期输出：['apple pie']

# 额外测试（验证多食物、非食物过滤场景）
query2 = 'i want to make yu xiang shredded pork and calorie < 30'
result5 = is_food_list(query2, qwen_model, qwen_tokenizer)
print(f"\n查询：{query2}")
print(f"提取的食物列表：{result5}")  # 预期输出：['orange', 'banana bread', 'milk']

query3 = 'i want to make kung pao chicken and low calorie food !'
result6 = is_food_list(query3, qwen_model, qwen_tokenizer)
print(f"\n查询：{query3}")
print(f"提取的食物列表：{result6}")  # 预期输出：['cake']


/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


提取出的食物关键词['protein-rich low-carb foods']
查询：recommend protein-rich low-carb foods
提取的食物列表：['protein-rich low-carb foods']
🎯 提取核心搭配：[i want to make yu xiang shredded pork and calorie < 30] → [make yu xiang shredded pork and calorie  30]
提取出的食物关键词['yu xiang shredded pork', 'calorie']

查询：i want to make yu xiang shredded pork and calorie < 30
提取的食物列表：['yu xiang shredded pork', 'calorie']
🎯 提取核心搭配：[i want to make kung pao chicken and low calorie food !] → [make kung pao chicken and low calorie food]
提取出的食物关键词['kung pao chicken', 'low calorie food']

查询：i want to make kung pao chicken and low calorie food !
提取的食物列表：['kung pao chicken', 'low calorie food']


In [22]:
#过滤函数，过滤出提取的哪些单词是食物
def filter_list(result_list, qwen_model, qwen_tokenizer):
    print(result_list)
    final_results = []
    for result in result_list:
        if is_food_query_final_solution(result, qwen_model, qwen_tokenizer) == 'YES':
            final_results.append(result)
    return final_results

test_list = filter_list(result6, qwen_model, qwen_tokenizer)
print(test_list)

['kung pao chicken', 'low calorie food']
🔍 是否是食物核心搭配判断：[kung pao chicken] → 输出：[YES]
🔍 是否是食物核心搭配判断：[low calorie food] → 输出：[YES]
['kung pao chicken', 'low calorie food']


In [23]:
#提取口味
def extract_taste_words(query):
    # 1. 定义所有口味相关关键词（包含基础和“偏XX”表达）
    taste_keywords = [
        "sour", "tart", "slightly sour",  # 酸/偏酸
        "sweet", "sugary", "slightly sweet",  # 甜/偏甜
        "bitter", "slightly bitter",  # 苦/偏苦
        "spicy", "hot", "peppery", "slightly spicy", "mildly spicy",  # 辣/偏辣
        "salty", "slightly salty"  # 咸/偏咸
    ]

    # 2. 清理特殊符号，统一转为小写
    query_clean = re.sub(r'[?.,!;<>≤≥]', '', query).strip().lower()

    # 3. 构建正则模式：优先匹配多词短语（如 slightly sour），避免被拆分为单字
    # 按关键词长度（空格数）排序，多词在前，单词在后
    sorted_keywords = sorted(taste_keywords, key=lambda x: len(x.split()), reverse=True)
    # 转义特殊字符，用|拼接，添加单词边界\b确保精准匹配
    pattern = r'\b(' + '|'.join(re.escape(key) for key in sorted_keywords) + r')\b'

    # 4. 提取所有匹配的口味词，去重并保持出现顺序
    matched_tastes = re.findall(pattern, query_clean)
    # 去重（保留首次出现的词）
    seen = set()
    result = []
    for taste in matched_tastes:
        if taste not in seen:
            seen.add(taste)
            result.append(taste)

    return result

In [24]:
# 去重
def del_same(arrlist):
    seen = set()
    result = []
    for arr in arrlist:
        if arr not in seen:
            seen.add(arr)
            result.append(arr)
    return result

In [25]:
def parse_conditions_first(query):
    field_mapping = {
        "calorie": "Calories", "calories": "Calories", "kcal": "Calories",
        "time": "TotalTime", "totaltime": "TotalTime", "duration": "TotalTime",
        "serving": "RecipeServings", "servings": "RecipeServings", "portion": "RecipeServings"
    }
    operator_mapping = {
        "<": "<", "<=": "<=", ">": ">", ">=": ">=", "=": "=", "==": "=",
        'less than':"<", 'greater than':">", 'less than or equal to':"<=",
        'greater than or equal to':">=", 'equal to':"="
    }

    pattern = r"(\w+\s*\w*)\s*([<>=]{1,2}|\w+\s+\w+(\s+\w+)?)\s*(\d+\.?\d*)"  # 补充匹配英文运算符（如less than）
    matches = re.findall(pattern, query.lower())
    conditions = []

    for match in matches:
        # 修正：match结构因补充英文运算符变了，取第0个为关键词、第1个为运算符、第3个为数值
        user_filed, user_op, _, value = match
        user_filed_clean = user_filed.replace(" ", "")
        meta_field = None
        for key in field_mapping:
            if key in user_filed_clean:
                meta_field = field_mapping[key]
                break
        if not meta_field:
            continue
        # 处理英文运算符（如"less than"）
        chroma_op = operator_mapping.get(user_op.strip(), None)
        if not chroma_op:
            continue
        conditions.append([meta_field, chroma_op, str(value)])  # 每个条件都是列表

    if len(conditions) == 0:
        return None
    elif len(conditions) >= 1:
        # 修复：初始化合并列表，循环添加后续条件+and
        merged_conditions = conditions[0]  # 先拿第一个条件（列表）
        for cond in conditions[1:]:  # 遍历剩余条件
            merged_conditions += ['and'] + cond  # 列表拼接：[A]+[and]+[B]
        return merged_conditions

In [26]:
#存放RAG检索后的数组
rag_lists = []
rag_name_lists = []

In [27]:
#测试版
prompt_template_final_fix2 = """
# 立即执行：基于检索结果推荐食谱（理由需自然改写,理由需包含检索数据中的 1 个具体食材或步骤。禁止生硬复制）
用户查询（query）：{{query}}
你的唯一任务：
1. 从下方3条检索结果中选distance≤0.4的推荐（都符合）；
2. 每个推荐必须包含：【食谱名（meta.Name）】、热量（meta.Calories）、总时间（meta.TotalTime）、口味(taste)；
3. 重点：推荐理由需满足以下2点（这是关键）：
   a. 信息来源：基于检索结果的text字段（如食材、步骤特点、口感相关描述），禁止编造任何未提及的内容（如没提“低糖”就不能说）；
   b. 语言创作：不能生硬复制text原文，要把text信息和用户需求（香蕉、时间<30分钟）结合，用自然的英文重新组织（比如把“premash the banana”改成“you can premash the banana in advance for easier mixing”）；
4. 最终回答用英文,只输出推荐列表（至少1个最多3个,尽量多推荐），不输出模板文字或多余内容。
5. 请根据菜谱名来判断这道菜的口味如何(sour, sweet, bitter, spicy, salty)
6. RAG检索结果只是参考，根据实际需要进行参考然后回答问题，不一定非要参照RAG结果



### 可用的RAG检索结果(若不符合用户提问的菜谱名(Name)，你可以自己编写)
{{retrieved_results}}

### 输出格式（理由必须自然改写，按这个结构来输出，不可省略任何参数）
仅输出JSON数组；
请按照以下JSON格式输出(不可以遗漏任何一个花括号！不要遗漏任何一个花括号！)：
[
 {
    "doc_id": {doc_id},
    "Name": {Name},
    "Calories": {Calories},
    "Total Time": {TotalTime},
    "Reason": "你自己编写的详细且令人信服的理由可以结合时间(TotalTime)和卡路里(Calories)来进行说明",
    "taste": "你自己判定的口味（口味有：sour, sweet, bitter, spicy, salty, slightly sour, slightly sweet, slightly bitter, slightly spicy, slightly salty）",
  },
]

"""

In [28]:
# 现在的根据食材推荐的最终输出
def rag_recipe_qa_fixed(user_query, model, tokenizer, top_k=3):
    """
    修复后的完整RAG问答函数：解决解码空白问题
    """
    print(f"🔍 正在处理查询：{user_query}")
    # 1. 检索结果（不变）
    #提取一句话中食物的单词
    test_list = extract_taste_words(user_query)
    result_list = is_food_list(user_query, model, tokenizer)
    filtered_results = del_same(test_list+result_list)
    print(f"过滤后的数组{filtered_results}")
    querystr = ' '.join(filtered_results)
    print(f"过滤后的结果{querystr}")
    retrieved_results = rag_retrieve(query=querystr, top_k=top_k)

    for i in range(len(retrieved_results)):
        rag_name_lists.append(retrieved_results[i]['meta']['Name'])
        rag_lists.append(retrieved_results[i])
    if not retrieved_results:
        return "抱歉，暂时没有找到符合您需求的美食"

    # 2. 拼接提示词（不变）
    final_prompt = build_recipe_prompt(
        query=user_query,
        retrieved_results=retrieved_results,
        prompt_template=prompt_template_final_fix2
    )
    # 3. 模型生成：关键修改——增大max_new_tokens，关闭截断
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [   {'role':"system",'content':'你是一个专业的厨师，向用户推荐你的食谱, 并且你能详细的有逻辑的描述你的理由'},
                {"role": "user", "content": final_prompt}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        # 关键修改：max_new_tokens设为800（足够生成完整推荐），加eos_token_id避免无意义生成
        outputs = model.generate(
            inputs,
            max_new_tokens=2000,  # 从500增至800，确保生成完整
            min_new_tokens=50,   # 强制生成至少50个token（避免太短）
            temperature=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 遇到结束符停止，避免冗余
            attention_mask=torch.ones_like(inputs)
        )

    # 修复解码：即使有特殊token，也先保留原始文本再过滤
    final_answer = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    return (final_answer,rag_lists) if final_answer.strip() else "模型已生成内容，但需调整解码逻辑（见原始内容）"

In [29]:
user_query = "i have milk and apple and eggs, which food can i make?"
prompt_template = prompt_template_final_fix2
rag_lists = []
rag_name_lists = []

# 2. 调用完整RAG函数生成回答
# （model、tokenizer、prompt_template都是你之前加载/定义好的，直接用）
"""final_answer = rag_recipe_qa(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
    prompt_template=prompt_template,
    top_k=3
)"""
final_answer = rag_recipe_qa_fixed(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
    top_k=3
)


🔍 正在处理查询：i have milk and apple and eggs, which food can i make?
提取出的食物关键词['milk', 'apple', 'eggs']
过滤后的数组['milk', 'apple', 'eggs']
过滤后的结果milk apple eggs


In [30]:
#设置提示词
prompt_what_do_you_mean="""
用户输入: {{user_query}}

# 任务
判定用户意图：
- 制作意图（输出true）：仅当用户明确询问“如何做某食物”的具体步骤、方法、流程（必须表达“求教程”的诉求），且含以下制作信号：how to、process、steps、way to、explain how、show me the steps、what steps、prep；
- 推荐意图（输出false）：以下所有情况均为推荐，即使含make/cook也返回false：
  1. 无制作意图（未求步骤），仅询问“选什么食物”“用现有食材能做什么”；
  2. 含推荐信号：need dishes、what should i make、what can i cook、any、recommend、suggest、some、i have（食材清单）；
  3. 无制作信号。

# 关键区分（必看）
- 算true：用户问“怎么做”（如“how to make cake”“what steps to cook rice”）；
- 算false：用户问“做什么”（如“I have eggs, what can I make?”“what should I cook for dinner”）。

# 注意事项
1. 仅输出true或false（无其他内容）；
2. 核心优先级：意图＞关键词（哪怕含make/cook，只要是问“做什么”，就返回false）。
"""
#判定用户意图（第二层滤网）
def whatDoYouMean(user_query,model,tokenizer):
    """
    判定用户意图，是直接询问食谱还是做食谱推荐？
    params:
    user_query:用户问题
    model:就是qwen_mode
    tokenizer: 就是qwen_tokenizer
    final_prompt: 提示词
    """
    final_prompt = prompt_what_do_you_mean.replace("{{user_query}}", user_query)
    # 模型生成：关键修改——增大max_new_tokens，关闭截断
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [   {'role':"system",'content':'你是专业餐厅接待员，精准判断用户是否在求“制作步骤教程”，否则都是推荐需求'},
                {'role':"system","content":"制作词+单个菜品→true；制作词+菜系大类→false；无制作词→false"},
                {"role": "user", "content": final_prompt}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        # 关键修改：max_new_tokens设为800（足够生成完整推荐），加eos_token_id避免无意义生成
        outputs = model.generate(
            inputs,
            max_new_tokens=10,  # 只需要输出true/false，10个token足够，避免多余内容
            temperature=0.0,    # 极低温度：让模型更“保守”，优先选高概率答案
            do_sample=False,    # 关闭采样：强制按最大概率输出（彻底消除随机性）
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones_like(inputs)
        )

    # 修复解码：即使有特殊token，也先保留原始文本再过滤
    final_answer = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    final_answer = final_answer.strip().lower()  # 统一小写，避免True/TRUE等格式问题

    """    # 规则兜底（仅处理模型输出无效的情况）
    if final_answer in ['true', 'false']:
        return final_answer
    else:
        # 极端情况：模型输出乱码，按“无教程诉求”返回false
        return 'false'"""

   # 核心修正：精准关键词库
    user_query_lower = user_query.lower()
    # 1. 制作信号（不变）
    make_signals = {'how to', 'make', 'cook', 'process', 'prepare', 'steps', 'way to', 'explain how', 'show me the steps', 'what steps', 'prep'}
    # 2. 推荐信号（不变）
    recommend_signals = {
        'what should i make', 'what can i cook', 'what foods are good for',
        'don’t take too long to make', 'need dishes', 'i need something',
        'any', 'recommend', 'suggest', 'some', 'which foods are suitable for',
        'i have', 'what can i make', 'what should i make', 'any', 'recommend', 'suggest', 'some'
    }
    # 3. 修正：仅保留“菜系大类关键词”（去掉容易误伤的“dishes”“soups”等）
    cuisine_big_category = {
        'cuisine', 'gluten-free lunch', 'fast food', 'low calorie foods',
        'healthy meal', 'street food'  # 只针对“大类”，不包含具体菜品相关词
    }

    # 最终判定逻辑（精准匹配）
    has_make = any(s in user_query_lower for s in make_signals)
    has_recommend = any(s in user_query_lower for s in recommend_signals)
    has_big_category = any(cat in user_query_lower for cat in cuisine_big_category)

    if has_make and not has_recommend and not has_big_category:
        # 制作词 + 无推荐信号 + 非大类 → true（制作单个菜品）
        return 'true'
    elif has_make and has_big_category:
        # 制作词 + 大类 → false（推荐）
        return 'false'
    elif has_recommend:
        # 有推荐信号 → false
        return 'false'
    else:
        # 无制作词 → false
        return 'false'


In [31]:
#user_query = "Do you know how to make Sichuan cuisine?" # false 正确
#user_query = "Can you tell me how to cook mapo tofu?" # true 正确
#user_query = "Suggest some quick breakfast options for work." # false 正确
#user_query = "What's the process to make Caesar salad?" # true
#user_query = "Recommend 2 spicy Sichuan dishes for dinner." # false 正确
#user_query = "How do I prepare grilled salmon?"  # true 正确
#user_query = "What are some healthy low-calorie foods I can make?"  # false 正确
#user_query = "Could you show me the steps to bake chocolate cookies?"  # true 正确
#user_query = "Give me some traditional Italian pasta recommendations."  # false 正确

#第二批测试
#user_query = "What's the way to make Japanese sushi rolls?" #true 正确
#user_query = "Recommend a good recipe for tomato soup." #false 正确
#user_query = "How do you cook Mediterranean seafood dishes?" # true 正确
#user_query = "What should I make for a vegan dinner party?" # false 正确
#user_query = "how to bake a cheesecake?" # true 正确
#user_query = "What are some variations of pizza I can try?" # false 正确
#user_query = "Explain how to prepare a healthy meal." # false 正确
#user_query = "Tell me the steps for making curry." # true 正确
#user_query = "do you know any low calorie foods?" # false 正确
#user_query = "Do you know which foods are suitable for weight loss" #false 正确

#第三批测试
#user_query = "What can I cook for a gluten-free lunch?" # false 正确
#user_query = "Explain the way to prepare street food snacks." # false 正确
#user_query = "Any ideas for easy dinner recipes?" # false 正确
#user_query = 'ice cream' # false 正确
#user_query = "kung pao chicken" # false 正确
#user_query = 'fast food' # false 正确
#user_query = "What foods are good for meal prep on Sundays?" # false 正确
#user_query = "Can you suggest how to grill perfect steaks?" # false 正确 （你能建议一下如何烤出完美的牛排吗？）
#user_query = "chinese food" # false 正确

# 第四批测试
#user_query = "how to make vegan brownies?" # true 正确
#user_query = "How to prep overnight oats quickly?" # true 正确
#user_query = "Which dishes are recommended for a picnic?" # false 正确
#user_query = "I don’t know how to cook tempura shrimp, can you help?" # true 正确
#user_query = "I need something for kids’ lunch, any easy options?" # false 正确
#user_query = "spicy noodles that don’t take too long to make" # false 正确

# final
#user_query = "What's the process to cook vegan soups?" # true 正确
#user_query = "I need dishes that prepare quickly for guests" # false 正确
#user_query = "Explain how to make healthy snacks for kids" # true 正确
#user_query = "How do you cook crispy fried chicken wings?" # true 正确
#user_query = "Any recipes that don’t require much cooking time?" # false 正确
#user_query = "How to prep a charcuterie board step by step?" # true 正确
#user_query = "Can you explain how to make matcha latte?" # true 正确
#user_query = "Do you have any weight loss meals?" # false 正确

#new final
#user_query = "I have milk, vegetables, potatoes, apples, bananas, and flour. What dishes can I make?" #false 正确
#user_query = "I have milk, vegetables... What dishes can I make?" # false 正确
user_query = "What can I cook for a gluten-free lunch?" # false 正确
#user_query = "how to make cake" # true 正确
#user_query = "how to make Sichuan cuisine" # false 正确
#user_query = 'how to make Chongqing hot pot' # true 正确
#user_query = 'food for picnic' #false 正确
#user_query = "what kind of food can i take to school?" #false 正确
#user_query = "low cal food" #false 正确
#user_query = 'spicy food' #false 正确
#user_query= 'I have bread' #false 正确
#user_query = 'Recommend a good recipe for tomato soup.' # false 正确
#user_query = 'how to make kung pao chicken?'

final_answer = whatDoYouMean(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
)

print(f"模型最终回答{final_answer}")

模型最终回答false


In [32]:
#最后滤网的提示词
final_mean_propmt = """
输入: {{user_query}}
任务：仅输出1个标签（ingredient/scene/cuisine_feature），多一个字都错！
判定规则：
1. 输入不含"i have" → 绝对不能输出ingredient
2. 输入是“在某个地点/活动吃什么” → 输出scene（比如ride/journey/visit都是活动）
3. 其他情况 → 输出cuisine_feature
示例：
- "food for a hot air balloon ride" → scene
- "recommend gluten-free meals" → cuisine_feature
仅输出标签，无任何额外内容！
"""


def FinalMean(user_query,model,tokenizer):
    """
    判定用户意图，是根据食材来推荐食谱还是根据菜系或特点来推荐食谱还是根据特殊场景来推荐？
    params:
    user_query:用户问题
    model:就是qwen_mode
    tokenizer: 就是qwen_tokenizer
    """
    user_query_lower = user_query.lower()
    final_prompt = final_mean_propmt.replace("{{user_query}}", user_query)

    # 步骤1：规则优先（原逻辑不变）
    if "i have" in user_query_lower:
        return "ingredient"
    scene_keywords = {
        'school', 'outside', 'picnic', 'office', 'party','camping', 'road trip','hiking trip',
        'late-night study session', 'boat trip', 'weekend getaway', 'rooftop dinner',
        'dormitory cooking', 'backpacking', 'music festival', 'early morning commutes',
        'hiking', 'bbq', 'camping trip', 'study session', 'getaway', 'rooftop', 'dormitory', 'overtime',
        'exercise', 'workout', 'beach picnic', 'family picnic','dormitories','fishing trip', 'meeting',
        'gym'
    }
    if any(keyword in user_query_lower for keyword in scene_keywords):
        return "scene"
    cuisine_keywords = {'chinese', 'low cal', 'low calorie', 'fast food', 'spicy', 'weight loss',
                       'vegan', 'no-cook', 'easy-to-carry', 'gluten-free', 'authentic', 'nutritious',
                       'easy-to-make', 'traditional', 'protein-rich', 'low-carb', 'mild-flavored', 'cuisine'
                       'recipe', 'prepare', 'quick', 'creamy', 'suitable for eating after'}
    if any(keyword in user_query_lower for keyword in cuisine_keywords):
        return "cuisine_feature"

    # 步骤2：模型调用（原逻辑不变）
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [
                {'role': "system", 'content': """
                你是只会输出3个标签的机器，必须遵守：
                    1. 只输出1个标签，绝不能输出2个；
                    2. 不含"i have"，绝对不能输出ingredient；
                    3. 输出后立刻停止，不加任何字、符号、换行。
                    """
                },
                {"role": "user", "content": final_prompt}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        outputs = model.generate(
            inputs,
            max_new_tokens=5,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # 步骤3：优化输出处理（关键修改：格式清洗+判定纠错）
    final_answer = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).strip()

    # 3.1 格式清洗：先去除换行、###、空格（保留字母，包括output后缀）
    clean_answer = final_answer.replace('\n', '').replace('###', '').replace(' ', '').lower()

    #截断犯人的子串
    if 'output' in clean_answer:
        clean_answer = clean_answer.replace('output','')

    print(f"大模型输出{clean_answer}")
    # 3.2 标签提取：兼容“标签+output”格式，精准匹配核心标签
    valid_labels = {'ingredient', 'cuisine_feature', 'scene'}
    correct_label = None
    # 遍历有效标签，只要clean_answer包含标签（不管后面有没有output），就提取
    for label in valid_labels:
        if label in clean_answer:
            correct_label = label
            break

    # 3.3 最终校验（你的简化逻辑不变）
    if correct_label:
        if correct_label == 'ingredient':
            return "ingredient"  # 无i have，强制修正
        if correct_label == 'scene':
            return 'scene'
        return correct_label
    else:
        return "cuisine_feature"

In [33]:
#第一批数据测试（大模型版）
#user_query = "I have milk, vegetables, potatoes, apples, bananas, and flour. What dishes can I make?" #ingredient 直接规则过滤正确
#user_query = "vegetables... What dishes can I make?" #ingredient 双正确
#user_query = "What can I cook for a gluten-free lunch?" # cuisine_feature 直接规则过滤正确
#user_query = "how to make Sichuan cuisine" # cuisine_feature 双正确
#user_query = 'food for picnic' #scene 直接规则过滤正确
#user_query = "what kind of food can i take to school?" #scene 直接规则过滤正确
#user_query = "low cal food" #cuisine_feature 直接规则过滤正确
#user_query = 'spicy food' #cuisine_feature 直接规则过滤正确
#user_query = "Recommend a good recipe for tomato soup." #cuisine_feature 最终通过所有规则过滤，大模型误判scene导致最终结果scene
#user_query = "What should I make for a vegan dinner party?" # scene 直接规则过滤正确
#user_query = "What are some variations of pizza I can try?" # cuisine_feature 双正确
#user_query = "Explain how to prepare a healthy meal." # cuisine_feature 最终通过所有规则过滤，大模型误判scene导致最终结果scene
#user_query = "do you know any low calorie foods?" # cuisine_feature 直接规则过滤正确
#user_query = "Do you know which foods are suitable for weight loss" #cuisine_feature 直接规则过滤正确

#第二批数据测试
#user_query = "I have eggs, what food can I take to school?"  # 预期: ingredient 直接规则过滤正确
#user_query = "food for office lunch"  # 预期: scene 直接规则过滤正确
#user_query = "spicy low cal chinese food"  # 预期: cuisine_feature 直接规则过滤正确
#user_query = "what to eat when camping?"  # 预期: scene 直接规则过滤正确
#user_query = "I have chicken, what weight loss food can I take to gym?"  # 预期: ingredient 直接规则过滤正确
#user_query = "french food for dinner party"  # 预期: scene 直接规则过滤正确
#user_query = "easy-to-make food"  # 预期: cuisine_feature 直接规则过滤正确
#user_query = "what food is good for a road trip?"  # 预期: scene 直接规则过滤正确
#user_query = "I have rice, recommend spicy fast food I can make"  # 预期: ingredient 直接规则过滤正确
#user_query = "traditional japanese food"  # 预期: cuisine_feature 双正确
#user_query = "I have bread"  # 预期: ingredient 直接规则过滤正确
#user_query = "I have cheese, what low cal food for picnic?"  # 预期: ingredient 直接规则过滤正确

#第三批数据测试
#user_query = 'What kind of food is suitable for weight loss?' # cuisine_feature 直接规则过滤正确
#user_query = "what to eat when camping?"  # 预期: scene 直接规则过滤正确
#user_query = "what food is good for a road trip?"  # 预期: scene 直接规则过滤正确
#user_query = 'Recommend a few Yunnan dishes' # cuisine_feature 双正确
#user_query = "what food is suitable for a hiking trip?"  # 预期: scene 直接规则过滤正确
#user_query = "Low calorie food suitable for camping and barbecue"  # 预期: scene 直接规则过滤正确
#user_query = "I have pasta, what to eat for a beach picnic?"  # 预期: ingredient 直接规则过滤正确
#user_query = "food for a late-night study session"  # 预期: scene 直接规则过滤正确
#user_query = "spicy food for a road trip and camping"  # 预期: scene 直接规则过滤正确
user_query = "I have chicken breast meat, which is suitable for eating after exercising"  # 预期: ingredient 直接规则过滤正确
#user_query = "which is suitable for eating after exercising" # 预期: cuisine_feature 最终通过所有规则过滤，大模型误判scene导致最终结果scene
#user_query = "no-cook food for a boat trip"  # 预期: scene 直接规则过滤正确
#user_query = "what to pack for a weekend getaway?"  # 预期: scene 直接规则过滤正确
#user_query = "I have oats, what food is good for early morning commutes?"  # 预期: ingredient 直接规则过滤正确
#user_query = "thai food for a rooftop dinner"  # 预期: scene 直接规则过滤正确
#user_query = "Recommend fast and light food suitable for dormitory cooking"  # 预期: scene 直接规则过滤正确
#user_query = "food for backpacking"  # 预期: scene 直接规则过滤正确
#user_query = "I have tofu, what vegan food for a family picnic?"  # 预期: ingredient 直接规则过滤正确
#user_query = "easy-to-carry food for a music festival"  # 预期: scene 直接规则过滤正确
#user_query = "Chinese fast food suitable for office overtime"  # 预期: scene 直接规则过滤正确
#user_query = "food for a hiking adventure"  # 预期: scene 直接规则过滤正确
#user_query = "what to eat during a study session?"  # 预期: scene 直接规则过滤正确
#user_query = "no-cook food for a beach getaway"  # 预期: scene 直接规则过滤正确
#user_query = "suitable for dormitory late-night cooking"  # 预期: scene 直接规则过滤正确

# 第四批测试用例（含预期结果）
# 1. 核心场景词变形（后缀/同义词测试）
#user_query = "what food fits a hiking adventure?"  # 预期: scene（核心词hiking）直接规则过滤正确
#user_query = "Simple food suitable for staying up late and cooking in dormitories"  # 预期: scene（核心词dormitory）直接规则过滤正确
#user_query = "I have quinoa, what to eat for a mountain getaway?"  # 预期: ingredient（含i have）直接规则过滤正确
#user_query = "what to bring for a day trip to the lake?"  # 预期: scene（核心词trip） 双正确
#user_query = "food for hiking, camping and beach picnics"  # 预期: scene 直接规则过滤正确
#user_query = "non-spicy food for a music festival crowd"  # 预期: scene（核心词music festival） 直接规则过滤正确
#user_query = "Food suitable for long-distance self driving without heating"  # 预期: scene（核心词自驾/road trip） 双正确
#user_query = "snacks for backpacking trips"  # 预期: scene（核心词backpacking） 直接规则过滤正确
#user_query = "I have whole wheat bread, a food suitable for commuting"  # 预期: ingredient（含我有） 直接规则过滤正确
#user_query = "korean food for a rooftop party with friends"  # 预期: scene（核心词rooftop） 直接规则过滤正确
#user_query = "nutritious food for a long-distance cycling trip"  # 预期: cuisine_feature（核心词cycling trip） 双正确
#user_query = "what to eat when staying in a hotel?"  # 预期: scene（核心词hotel） 双正确
#user_query = "I have spinach, what vegan easy-to-carry food for a festival?"  # 预期: ingredient 直接规则过滤正确
#user_query = "A light salad suitable for office team building"  # 预期: scene（核心词office） 直接规则过滤正确
#user_query = "food for a hot air balloon ride"  # 预期: scene（核心词ride/trip场景延伸） 双正确
#user_query = "Recommend a few delicious foods at the amusement park" # scene 双正确

final_answer = FinalMean(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
)
print(f"{final_answer}")

ingredient


In [34]:
# 侥幸测试 第一批
# 这次的双正确是大模型和最后return结果都正确
# 1. 小众场景（无规则关键词，需大模型识别场景语义）
#user_query = "what food to bring for a hot air balloon ride?"  # 预期: scene（活动场景） 双正确
#user_query = "what to eat during a long train journey?"  # 预期: scene（出行场景） 双正确
#user_query = "recommend gluten-free vegan meals"  # 预期: cuisine_feature（双特点，无场景） 直接规则过滤正确
#user_query = "snacks for a museum visit"  # 预期: scene 双正确
#user_query = "how to make authentic thai green curry?"  # 预期: cuisine_feature（菜系相关）双正确
#user_query = "no-oven food for a rooftop gathering"  # 预期: scene（场合场景） 直接规则过滤正确
#user_query = "nutritious meals for busy professionals"  # 预期: cuisine_feature 直接规则过滤正确
#user_query = "food for a weekend fishing trip"  # 预期: scene（活动场景） 双正确
#user_query = "quick meals for early morning yoga"  # 预期: cuisine_feature（特点+活动，无场景词） 直接规则过滤
#user_query = "what to eat when staying in a mountain cabin?"  # 预期: scene（地点场景）双正确
#user_query = "mild-flavored korean side dishes"  # 预期: cuisine_feature（菜系+特点） 直接规则过滤正确
#user_query = "food for a cave exploration trip"  # 预期: scene（活动场景） 双正确
#user_query = "recommend protein-rich low-carb foods"  # 预期: cuisine_feature（双特点） 直接规则过滤正确
#user_query = "finger food for a book club meeting"  # 预期: scene（场合场景） 双正确
#user_query = "how to make creamy mushroom pasta?"  # 预期: cuisine_feature（具体菜品，无场景） 直接规则过滤正确

# 最终测试用例（9个，覆盖全场景）
# 1. ingredient（含i have，规则直接过滤）
#user_query = "I have chicken breast meat, which is suitable for eating after exercising"  # 预期: ingredient 直接规则过滤正确
# 2. scene（含明确场景词，规则直接过滤）
#user_query = "food for a hiking adventure"  # 预期: scene 正确
# 3. cuisine_feature（含常见特点词，规则直接过滤）
#user_query = "do you know any low calorie foods?"  # 预期: cuisine_feature 直接规则过滤正确
# 4. 优先级冲突（含i have+scene，规则优先ingredient）
#user_query = "I have cheese, what low cal food for picnic?"  # 预期: ingredient 正确
# 5. 规则未覆盖（模糊场景，需大模型补位）
#user_query = "what to eat when staying in a mountain cabin?"  # 预期: scene 双正确
# 6. 多特点叠加（无场景，规则直接过滤）
#user_query = "spicy low cal chinese food"  # 预期: cuisine_feature 直接规则过滤正确
# 7. 大模型易误判（无规则词+特点，需模型识别）
#user_query = "recommend protein-rich low-carb foods"  # 预期: cuisine_feature 大模型误判ingredient 导致最终结果误判 ingredient
# 8. 场景词变形（规则覆盖，验证关键词灵活性）
#user_query = "what to bring for a day trip to the lake?"  # 预期: scene 双正确
# 9. 纯菜系（无场景，规则直接过滤）
#user_query = "Recommend a few Yunnan dishes"  # 预期: cuisine_feature 双正确

final_answer = FinalMean(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
)
print(f"{final_answer}")

ingredient


In [14]:
import re
import torch

JUDGE_PROMPT_TEMPLATE = """
用户输入: {{user_query}}

# 任务
仅输出YES或NO，判定是否为美食相关话题（包括：可食用的食物/食材/菜系/料理/昆虫，制作方法，饮食需求/营养推荐，食材搭配）。

# 关键示例（必须参考）
✅ YES：
- how to make Sichuan cuisine（川菜是食物）
- Italian dishes（意大利菜是食物）
- How to cook insect（昆虫可食用）
- make ice cream（制作食物）
- bake pizza（烤披萨）
- 30 minute kwai（快手菜）
- high-protein low-carb foods（高蛋白低碳水食物推荐）
- vegan dishes under 300 calories（低卡素食推荐）
- food for post-workout recovery（健身后食物）

❌ NO：
- 不可食用的人/物（如trump、phone、stone、rock）
- 与食物无关的内容（如how are you）
- 与食物无关的闲聊/情绪发泄（无食物核心词）

只输出YES或NO！
"""

def is_food_query_final_solution(userquery, model, tokenizer):
    # 步骤1：清理情绪词/脏话，保留核心语义（保留连字符）
    dirty_words = {'fuck', 'ass', 'shit', 'bitch', 'hate', 'omfg'}
    userquery_clean = userquery.strip().lower()
    for word in dirty_words:
        userquery_clean = re.sub(rf'\b{word}\b', '', userquery_clean)
    # 关键：保留连字符（不替换），只清理多余空格
    userquery_clean = re.sub(r'\s+', ' ', userquery_clean).strip()
    # 步骤2：提取“完整词汇”（包括连字符复合词，如high-protein）
    words = re.findall(r'\b[\w-]+\b', userquery_clean)  # 用正则提取带连字符的单词

    # 步骤3：动作+食物对象规则（支持复合词）
    food_actions = {'make', 'cook', 'eat', 'prepare', 'bake', 'fry', 'crave', 'recommend', 'suggest', 'need'}
    food_objects = {
        'ice cream', 'cake', 'dish', 'food', 'meal', 'snack', 'insect', 'bug',
        'pizza', 'rice', 'vegetables', 'quinoa', 'protein', 'carb', 'calories', 'vegan',
        'high-protein', 'low-carb', 'post-workout'  # 新增复合词
    }
    # 检查是否有动作+食物对象（包括复合词）
    has_food_action = any(action in words for action in food_actions)
    has_food_object = any(obj in words or obj.replace('-', ' ') in userquery_clean for obj in food_objects)
    if has_food_action and has_food_object:
        print(f"🔍 动作+食物对象匹配：[{userquery}] → 输出：[YES]")
        return 'YES'

    # 步骤4：弱规则过滤（支持复合词匹配）
    clear_food_words = {
        'food', 'meal', 'dish', 'snack', 'ingredient', 'recipe',
        'cuisine', 'cuisines', 'dishes',
        'hungry', 'starving', 'craving',
        'cheesecake', 'chess cake', 'milkshake', 'milk shake',
        'insect', 'edible insect', 'bug', 'edible bug', 'quinoa',
        # 营养/饮食复合词
        'protein', 'carb', 'carbs', 'calorie', 'calories',
        'high-protein', 'low-carb', 'post-workout', 'recovery',
        'vegan', 'vegetarian', 'keto', 'diet', 'weight loss',
        'milk'
    }
    clear_non_food_words = {
        'trump', 'biden', 'phone', 'car', 'money', 'computer', 'book', 'stone', 'rock', 'dog',
        'toy car', 'homework', 'doll', 'video game', 'pencil', 'crayon', 'weapon', 'weapons', 'gta5'
    }

    # 关键：先检查非食物词（完整匹配，包括复合词）
    if any(non_food in words for non_food in clear_non_food_words):
        print(f"🔍 明确非食物词匹配：[{userquery}] → 输出：[NO]")
        return 'NO'
    # 再检查食物词（支持复合词或空格分隔，如high protein）
    if any(food in words or food.replace('-', ' ') in userquery_clean for food in clear_food_words):
        print(f"🔍 明确食物词匹配：[{userquery}] → 输出：[YES]")
        return 'YES'

    # 步骤5：模型判定（提示词已优化）
    judge_prompt = JUDGE_PROMPT_TEMPLATE.replace("{{user_query}}", userquery_clean)
    print("进入模型判定阶段")
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [
                {'role':"system", "content": '忽略脏话、情绪词、无关事件，只看核心语义是否为食物/饮食需求/营养推荐；你只能输出YES/NO'},
                {"role": "user", "content": judge_prompt},
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        outputs = model.generate(
            inputs,
            max_new_tokens=2,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    raw_output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).strip()
    clean_output = re.sub(r'[。.\s]', '', raw_output).upper()
    print(f"🔍 模型核心判定：[{userquery}] → 输出：[{clean_output}]")
    return clean_output

In [15]:
# 开始第一批测试
#query = "I have milk, vegetables, potatoes, apples, bananas, and flour. What dishes can I make?" # 期待YES 正确
#query = "I have eggs, what food can I take to school?" # 期待YES 正确
#query = 'What kind of food is suitable for weight loss?' # 期待YES 正确
#query = 'What to eat when staying in a mountain cabin?' # 期待YES 正确
#query = 'Recommend a few Yunnan dishes' # 期待YES 正确
#query = 'kung pao chicken' # 期待YES 正确
#query = 'milk' # 期待YES 正确
#query = 'mapo tofu' # 期待YES 正确
#query = 'ice cream' # 期待YES 正确
#query = 'apple pie' # 期待YES 正确
#query = 'how to cook fish' # 期待YES 正确
#query = 'Low calorie food suitable for camping and barbecue' # 期待YES 正确
#query = 'easy-to-carry food for a music festival' # 期待YES 误判
#query = "I have quinoa, what to eat for a mountain getaway?"  # 期待YES 正确
#query = 'I am craving something sweet' # 期待YES 正确
#query = 'how to make money in GTA' # 期待NO 正确
#query = 'i want to cook a love food' # 期待YES 正确
#query= 'How to cook stone'# 期待NO 正确
#query = 'How to cook insect' # 期待YES 正确
#query = 'fuck you! now! move your ass to make a ice cream!!!!' # 期待YES 正确
#query = 'recommend some spicy food'  # 期待YES 正确

# 侥幸测试
# 1. 小众场景
#query = "what food to bring for a hot air balloon ride?"  # 期待YES 正确
#query = "what to eat during a long train journey?"  # 期待YES 正确
#query = "recommend gluten-free vegan meals"  # 期待YES 正确
#query = "snacks for a museum visit"  #期待YES 正确
#query = "how to make authentic thai green curry?"  # 期待YES 正确
#query = "no-oven food for a rooftop gathering"  # 期待YES 正确
#query = "nutritious meals for busy professionals"  # 期待YES 正确
#query = "food for a weekend fishing trip"  # 期待YES 正确
#query = "quick meals for early morning yoga"  # 期待YES 正确
query = "what to eat when staying in a mountain cabin?"  # 期待YES 正确
#query = "mild-flavored korean side dishes"  # 期待YES 正确
#query = "food for a cave exploration trip"  # 期待YES 正确
#query = "recommend protein-rich low-carb foods"  # 期待YES 误判
#query = "finger food for a book club meeting"  # 期待YES 误判
#query = "how to make creamy mushroom pasta?"  # 期待YES 正确
#query = "What can the protagonist eat to regain blood in GTA5???" # 期待NO 正确
#query = "What can I eat to nourish my brain?" # 期待YES 正确

# 第三批测试
#query = "OMFG I HATE YOU BUT MAKE ME A SPICY CHEESECAKE RIGHT NOW!!!"# 期待YES 正确
#query = "how to cook a rock with edible bug powder for hiking?" # 期待YES 正确
#query = "my dog ate my phone, should I bake gluten-free pizza to calm down?" # 期待YES 正确
#query = "Recommend 30 minute fast food suitable for office workers" # 期待YES 正确
#query = "What are the high protein snacks that can be eaten during the weight loss period" # 期待YES 正确
#query = "What dishes can I make with only rice and vegetables at home" # 期待YES 正确
#query = "What light and nutritious complementary foods are suitable for children who are picky eaters" # 期待YES 正确
#query = "What convenient food to bring for camping that doesn't require heating" # 期待YES 正确

#第四批，熊孩子专场
#query = "cook my toy car with a little bit of sugar!!!" # 期待NO 正确
#query = "eat my homework but add some cheese on top" # 期待NO 正确
#query = "what to make for my doll’s birthday party?" # 期待NO 正确
#query = "bake a phone cake with chocolate syrup" # 期待NO 误判
#query = "I’m hungry give me a video game to eat!!!" # 期待NO 正确
#query = "how to prepare a pencil snack for school?" # 期待NO 误判
#query = "craving something blue—let’s cook a crayon!" # 期待NO 正确
#query ='Can you recommend the weapons in GTA5' # 期待NO 正确

#final test
#query = "I have milk, vegetables, potatoes, apples, bananas, and flour. What dishes can I make?" #期待YES 正确
#query = "I have milk, vegetables... What dishes can I make?" #期待YES 正确
#query = "What can I cook for a gluten-free lunch?" #期待YES 正确
#query = "how to make cake" #期待YES 正确
#query = "how to make Sichuan cuisine" #期待YES 正确
#query = 'how to make Chongqing hot pot' #期待YES 正确
#query = 'food for picnic' #期待YES 正确
#query = "what kind of food can i take to school?"#期待YES 正确
#query = 'spicy food' #期待YES 正确
#query= 'I have bread' #期待YES 正确
#query = 'Recommend a good recipe for tomato soup.'#期待YES 正确
#query = 'what weight loss food can I take to gym?'
#query = 'What high-protein low-carb foods are good for post-workout recovery?'
#query = 'recommend some news paper'
query = 'do you know how to make kung pao chicken?'

is_food_query_final_solution(query, qwen_model, qwen_tokenizer)

/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and canno

进入模型判定阶段
🔍 模型核心判定：[do you know how to make kung pao chicken?] → 输出：[YES]


'YES'

In [37]:

# 高频菜系白名单（兜底用）
COMMON_CUISINES = {
    'chinese', 'sichuan', 'cantonese', 'yunnan', 'shandong', 'chongqing', 'xinjiang',
    'hunan', 'taiwanese', 'guizhou', 'macanese', 'tibetan',
    'japanese', 'italian', 'french', 'indian', 'mexican', 'thai', 'korean', 'american',
    'english', 'burmese', 'australian', 'new zealand', 'german', 'vietnamese',
    'korea', 'germany', 'vietnam'
}

prompt_template_cuisine = """
# 任务：从用户查询中提取**唯一1个核心菜系**（国家/地区/地方饮食名）
用户查询（query）：{user_query}

# 规则：
1. 优先提取第一个出现的核心菜系（如Hunan和Sichuan，先提hunan）
2. 仅输出1个小写单词，非菜系词/无菜系则输出空
3. 禁止输出xinjiang（除非查询含该词）、禁止输出任何符号/换行/乱码

# 输出格式：仅1个纯单词（无符号、无换行）或空，无任何额外内容！
"""

def extract_single_cuisine(userquery, qwen_model, qwen_tokenizer):
    user_input = userquery.lower()
    final_prompt = prompt_template_cuisine.replace("{user_query}", userquery)

    # 步骤1：大模型提取
    inputs = qwen_tokenizer.apply_chat_template(
        [
            {'role': "system", "content": '严格按规则提取，仅输出纯单词，无符号/换行/乱码，非菜系输出空'},
            {"role": "user", "content": final_prompt}
        ],
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(qwen_model.device)

    with torch.no_grad():
        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=4,
            temperature=0.0,
            do_sample=False,
            eos_token_id=qwen_tokenizer.eod_id
        )

    # 解码并强化清洗（核心修复）
    raw_output = qwen_tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip().lower()
    # 1. 仅保留字母（剔除数字、符号、空格）
    clean_output = re.sub(r'[^a-z]', '', raw_output)  # 重点：去掉所有非字母字符，包括空格
    # 2. 白名单校验：从清洗后的字符串中匹配完整菜系词（解决拼接错误）
    matched_cuisine = ""
    for cuisine in COMMON_CUISINES:
        if cuisine in clean_output:  # 例如从"chinesefamily"中匹配到"chinese"
            matched_cuisine = cuisine
            break  # 只取第一个匹配的
    # 3. 过滤违规词
    if matched_cuisine == 'xinjiang' and 'xinjiang' not in user_input:
        matched_cuisine = ""

    if matched_cuisine:
        print(f"大模型提取（清洗后）：{matched_cuisine}")
        return matched_cuisine

    # 步骤2：白名单兜底
    print(f"大模型未提取到，白名单兜底匹配")
    for cuisine in COMMON_CUISINES:
        if re.search(rf'\b{cuisine}\b', user_input):
            result = 'korean' if cuisine == 'korea' else 'vietnamese' if cuisine == 'vietnam' else 'german' if cuisine == 'germany' else cuisine
            print(f"白名单匹配：{result}")
            return result

    print(f"无匹配菜系：None")
    return None

In [38]:
# 测试
# 带菜系的测试用例
#query = 'how to make traditional chinese food' # 正确
#query = 'i want to make sichuan hot pot and some yunnan food'  # 正确
#query = 'i want to make Chongqing hot pot' # 正确
#query = 'i want to make some korea food, tell me what should i do?' # 正确
#query = 'I want to make a few Cantonese dishes' # 正确
#query = 'Can you teach me how to make a few English breakfasts' # 正确
#query = 'Recommend a few Yunnan dishes' # cuisine_feature 正确

# 不带菜系的测试用例，但是我之前设计的思维链判定它是走“菜系和特点推荐的”，这些用例的目的是不要提取出任何词
#query = "What can I cook for a gluten-free lunch?" # cuisine_feature ，正确返回None
#query = "Recommend a good recipe for tomato soup." #cuisine_feature, 正确返回None
#query = 'recommend a low cal food'  #cuisine_feature, 正确返回None

# 第三批测试数据： 绕过规则匹配白名单测试
#query = 'Recommend a few Xinjiang dishes' # 正确
#query = 'Recommend a few Burmese dishes' # 正确
#query = 'Recommend several dishes suitable for weight loss' # 大模型提取：healthy
#query = 'Recommend a few New Zealand dishes' # 大模型提取：new
#query = 'Recommend a few Australian dishes' # 正确
#query = 'Recommend a few German dishes' # 正确
#query = 'Recommend a few Guizhou dishes' # 正确

# 第四批，混合测试
#query = 'I want to cook Hunan spicy dishes and Sichuan food' # 正确
#query = 'Recommend Taiwanese snacks and Cantonese dim sum' # 正确
#query = 'Korean barbecue vs Japanese sushi, which is easier?' # 正确
#query = 'Vietnamese pho and Thai tom yum soup, teach me one' # 正确
#query = 'Recommend a few weight loss meals'
#query = "traditional japanese food"
#query = 'recommend some chinese food that is suitable for family picnic'

reslut = extract_single_cuisine(query, qwen_model, qwen_tokenizer)
print(reslut)

大模型未提取到，白名单兜底匹配
无匹配菜系：None
None


In [39]:
import re
import torch

prompt_template_feature = """
# 任务：从用户查询中抽象出食物特征，输出特征数组（无特征则输出空数组）
用户查询（query）：{user_query}

# 特征定义（满足任意一类即可）：
1. 口味：spicy、sweet、salty、sour、mild等描述味道的词
2. 属性：low calorie、high protein、fast、short time等描述食物特点的词/短语
3. 隐含特征：weight loss→low calorie、fitness meal→high protein、short preparation time→fast

# 强制规则：
1. 必须输出数组格式，特征用小写，短语用空格连接
2. 无特征则输出空数组，禁止输出任何额外内容
3. 特征不重复，数量1-3个即可（无需过多）

# 示例：
1. recommend some family food that feature is low calorie → ['low calorie' , 'family food']
2. Do you have any fitness meal recommendations? → ['high protein']
3. Recommend a sweet food with a short preparation time → ['sweet', 'fast']
4. recommend some spicy food → ['spicy']
5. What can I cook for dinner? → []
6. What kind of food is suitable for family picnic? → ['family food']
7. food for watching horror movies alone → ['finger food', 'easy to eat']

# 输出格式：严格按示例，仅数组！例如：['spicy', 'low calorie'] 或 []
"""

def extract_feature(userquery, qwen_model, qwen_tokenizer):
    final_prompt = prompt_template_feature.replace("{user_query}", userquery)

    # 大模型自主提取（无白名单干扰）
    inputs = qwen_tokenizer.apply_chat_template(
        [
            {'role': "system", "content": '严格按示例和规则提取特征，必须输出数组格式！'},
            {"role": "user", "content": final_prompt}
        ],
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(qwen_model.device)

    with torch.no_grad():
        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=30,  # 足够容纳3个特征的数组
            temperature=0.2,  # 适度随机性，保证输出
            do_sample=False,
            eos_token_id=qwen_tokenizer.eod_id
        )

    # 解析输出（只提取数组内的特征）
    raw_output = qwen_tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip().lower()
    # 提取引号内的所有特征
    features = re.findall(r'[\'\"]([a-z\-\s]+)[\'\"]', raw_output)
    # 清理（去空、去重）
    features = [feat.strip() for feat in features if feat.strip()]
    features = list(set(features))[:3]  # 最多保留3个，避免冗余

    print(f"提取的食物特征：{features or '[]'}")
    return features

In [40]:
# 第一批数据测试
#query = 'Recommend a few weight loss meals' # 提取的食物特征：['low calorie'] 正确
#query = 'recommend some spicy food' # 提取的食物特征：['spicy'] 正确
#query = 'recommend a chinese food that low calorie' # 提取的食物特征：['low calorie'] 正确
#query = 'recommend some fast food which is low cal' # 提取的食物特征：['low calorie', 'fast'] 正确
#query =  "spicy low cal chinese food"  # 提取的食物特征：['low calorie', 'spicy'] 正确
#query = 'Recommend a few Yunnan dishes' # 提取的食物特征：['yunnan dishes'] 正确
#query = 'low cal and spicy and fast food' # 提取的食物特征：['low calorie', 'spicy'] 少了fast
#query = 'Recommend a sweet food with a short preparation time' # 提取的食物特征：['fast', 'sweet'] 正确
#query = 'Do you have any fitness meal recommendations?' # 提取的食物特征：['fast', 'high protein'] 正确
#query = "Recommend a good recipe for tomato soup."  # 提取的食物特征：['tomato soup'] 正确
#query = "traditional japanese food"  # 提取的食物特征：['traditional', 'japanese'] 正确

# 第二批测试
#query = 'Recommend diet meals for office workers' #提取的食物特征：['low calorie', 'fast'] 正确，而且出乎意料的是，它抽象出了fast这个标签，很棒！
#query = 'I need nutritious food for post-workout recovery' #提取的食物特征：['high protein', 'fast', 'nutritious'] 正确，而且出乎意料的是，fast可有可无，不过抽象出来也行
#query = 'quick and healthy snacks for kids' # 提取的食物特征：['healthy', 'kids friendly', 'quick'] 完全命中
#query = 'vegan meals with no added sugar' #提取的食物特征：['no added sugar', 'vegan'] 完全命中
#query = 'spicy high protein low carb chinese food' #提取的食物特征：['spicy', 'low carb']少了high protein
#query = 'mild-flavored fast food for a road trip' #提取的食物特征：['mild-flavored', 'fast food'] 完全命中
#query = 'gluten-free sweet snacks with short preparation time' #提取的食物特征：['fast', 'sweet'] 少了gluten-free
#query ='salty and crispy finger food for a party' #提取的食物特征：['salty', 'finger food', 'crispy'] 全名中
#query = 'creamy dairy-free pasta recipe' #提取的食物特征：['creamy', 'dairy-free'] 全名中
#query = 'chewy vegan candy with natural ingredients' #提取的食物特征：['natural', 'low calorie', 'vegan'] 很棒，它还抽象出low calorie
#query = 'tangy pickled side dishes for rice' #提取的食物特征：['low calorie', 'spicy'] 抽象的很棒
#query = 'What food can I make with rice?' #提取的食物特征：['rice', 'low calorie'] 抽象出这些也还好，不是很影响
#query = 'food that’s easy to find in supermarkets' #提取的食物特征：['low calorie', 'fast'] low cal也还好，抽象出fast food就很棒
#query = 'What kind of food is suitable for taking family out for a picnic?' # 提取的食物特征：['suitable for family picnic'] 我其实更想它抽象为family food
#query = 'What kind of food is suitable for a person to eat while watching horror movies in a single apartment' # 提取的食物特征：['low calorie', 'spicy'] 不知道算不算好。。。。
#query = 'Recommend a few healthy breakfasts' # 提取的食物特征：['low calorie', 'breakfast', 'healthy']
#query = 'Recommend some weight loss meals' # 提取的食物特征：['fast', 'high protein', 'weight loss meals'] 正确
#query = "Recommend some delicious foods, don't be high in protein, I am allergic to high protein" # 提取的食物特征：['low calorie'] 正确
#query = 'Recommend several fitness foods' # 提取的食物特征：['short time', 'fast', 'high protein'] 正确
#query = 'Recommend several delicious foods suitable for eating in the dormitory' # 提取的食物特征：['fast', 'low calorie'] 正确
#query = 'Recommend several high calorie foods'  # 提取的食物特征：high calorie 正确
#query = 'Recommend several vegetable based delicacies'
query = 'what kind of food can I take to school?'



final_answer = FinalMean(
    user_query=query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
)
final_answer2 = whatDoYouMean(
    user_query=query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
)
print(final_answer)
print(final_answer2)
extract_feature(query, qwen_model, qwen_tokenizer)

scene
false


/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


提取的食物特征：['healthy eating', 'fast food', 'high protein']


['healthy eating', 'fast food', 'high protein']

In [41]:
#词句翻译
books = {
        # 核心热量/营养（保留原有+补充隐含词）
    'low calorie': ['low calorie', 'weight loss', 'loss weight', 'diet', 'slim', 'reduce weight', 'weight control', 'low cal' ],
    'high calorie': ['high calorie', 'high cal', 'energy-rich'],
    'high protein': ['high protein', 'protein-rich', 'fitness meal', 'post-workout', 'muscle gain'],
    'low protein': ['low protein', 'protein-poor'],
    'high carb': ['high carb', 'carb-rich'],
    'low carb': ['low carb', 'carb-poor', 'keto'],

    # 饮食限制/特殊需求（新增高频场景）
    'vegan': ['vegan', 'plant-based', 'no animal products'],
    'vegetarian': ['vegetarian', 'no meat'],
    'gluten-free': ['gluten-free', 'no gluten'],
    'dairy-free': ['dairy-free', 'no dairy', 'lactose-free'],
    'no sugar': ['no sugar', 'sugar-free', 'no added sugar'],

    # 场景/标签（补充家庭/聚会/便携等）
    'family food': ['family food', 'family meal', 'family picnic', 'for family'],
    'fast food': ['fast food', 'fast', 'short time', 'quick', 'easy to make', '30 minute', 'no-oven'],
    'finger food': ['finger food', 'party food', 'for party', 'snacks for gathering'],
    'picnic food': ['picnic food', 'for picnic', 'outdoor food'],
    'portable food': ['portable', 'easy to carry', 'travel food', 'road trip food'],
}

special_words={
    "low calorie" : 'calories < 250',
    "high calorie" : 'calories > 250',
    "family food": 'RecipeServings > 3',
    "fast food": 'TotalTime < 60',
}

#把特征翻译映射
def translate(feature_list):

    # 1. 收集所有映射表中的关键词（转小写）
    all_keywords = set()
    for keywords in books.values():
        all_keywords.update({kw.lower() for kw in keywords})

    # 2. 提取映射特征（用正则单词边界匹配，避免子串误判）
    mappers = []
    for std_label, keywords in books.items():
        for fea in feature_list:
            fea_lower = fea.lower()
            # 关键：用\b匹配单词边界，确保kw是独立单词
            if any(re.search(rf'\b{kw.lower()}\b', fea_lower) for kw in keywords):
                mappers.append(std_label)
                break  # 一个特征匹配一个标准标签即可，避免重复

    # 去重映射结果
    mappers = list(set(mappers))

    # 3. 特殊词替换
    for i in range(len(mappers)):
        for kw, v in special_words.items():
            if kw in mappers[i]:
                mappers[i] = v

    # 4. 保留未匹配特征（同样用正则单词边界判断）
    unmappers = []
    for feat in feature_list:
        feat_lower = feat.lower()
        is_unmatched = True
        for kw in all_keywords:
            if re.search(rf'\b{kw}\b', feat_lower):
                is_unmatched = False
                break
        if is_unmatched:
            unmappers.append(feat)

    result = mappers + unmappers

    return result

#合并cuisine和feature
def combine_cuisine_feature(feature_list,cuisine):
    #如果检测到仅仅只推荐特征
    if cuisine is None:
        strs = ' and '.join(feature_list)
        return strs

    mapper_list = [feat for feat in feature_list if not (cuisine in feat)]
    strs = ' and '.join(mapper_list)
    #如果仅仅只是推荐菜系
    if len(mapper_list) == 0:
        return cuisine + ' cuisine '

    #又有菜系又有特征
    cuisine = cuisine + ' that is '
    return cuisine + strs

#总合并函数——这是主函数
"""
该模块是菜系特点的总合并模块, 该模块包括:
    1. 提取菜系
    2. 提取特点
    3. 翻译特点
    4. 合并菜系特点
"""
def main_combine_cuisine_feature(userquery, qwen_model, qwen_tokenizer):
    cuisine = extract_single_cuisine(userquery, qwen_model, qwen_tokenizer)
    feature_list = extract_feature(userquery, qwen_model, qwen_tokenizer)
    translate_list = translate(feature_list)
    result = combine_cuisine_feature(translate_list,cuisine)
    if result == '':
        print(f"最终问题{userquery}")
        return userquery #如果没提取到问题，则返回最原始的问题
    print(f"最终问题{result}")
    return result

In [42]:
import random
#query = "traditional japanese food"  # 提取的食物特征：['traditional', 'japanese'] 正确
#query = 'What kind of food is suitable for a person to eat while watching horror movies in a single apartment' # 提取的食物特征：['low calorie', 'spicy'] 不知道算不算好。。。。
#query = "Recommend a few Yunnan dishes"
#query = 'recommend a low cal and family food'
#query = 'recommend some Sichuan cuisine'
#query = 'recommend some chinese food that is suitable for family picnic'
#query = 'recommend some korea food which is suitable for some one who want to loss weight'
#query = 'Recommend several vegetable based delicacies'
result = main_combine_cuisine_feature(query, qwen_model, qwen_tokenizer)
rags_list = rag_retrieve(result,8)


大模型提取（清洗后）：chinese
提取的食物特征：['healthy eating', 'fast food', 'high protein']
最终问题chinese that is TotalTime < 60 and high protein and healthy eating


In [43]:
import random
import json
#构建推荐提示词
prompt_template_recommend = """
# 立即执行：基于检索结果推荐食谱（理由需自然改写,理由需包含检索数据中的 1 个具体食材或步骤。禁止生硬复制）
用户查询（query）：{{query}}
你的唯一任务：
1. 每个推荐必须包含：【食谱名（meta.Name）】、热量（meta.Calories）、总时间（meta.TotalTime）、口味(taste)；
2. 重点：推荐理由需满足以下2点（这是关键）：
   a. 信息来源：基于检索结果的text字段（如食材、步骤特点、口感相关描述），禁止编造任何未提及的内容（如没提“低糖”就不能说）；
   b. 语言创作：不能生硬复制text原文，要把text信息和用户需求（香蕉、时间<30分钟）结合，用自然的英文重新组织（比如把“premash the banana”改成“you can premash the banana in advance for easier mixing”）；
3. 最终回答严格要求：
   - - 仅输出**1次**符合格式的JSON列表，**绝对不允许重复输出相同内容**，**不包含任何额外文字、解释、说明或模板残留内容**；
   - 推荐1-3个，直接输出JSON数组；
4. 请你根据菜谱简介来判断这道菜的口味如何(sour, sweet, bitter, spicy, salty)

### 可用的RAG检索结果(若不符合用户提问的菜谱，你可以根据你自己的数据库来推荐你的食谱, 不可以说不知道！请根据你个人数据库回答)
{{retrieved_results}}

### 输出格式（理由必须自然改写，按这个结构来输出，不可省略任何参数, 不要遗漏taste! 请你根据食谱信息判定taste）
仅输出1次JSON数组，无其他内容
请按照以下JSON格式输出(不可以遗漏任何一个花括号！不要遗漏任何一个花括号！)：
[
{
    "Name": {Name},
    "Calories": {Calories},
    "Total Time": {TotalTime},
    "Reason": "你自己编写的详细且令人信服的理由可以结合时间(TotalTime)和卡路里(Calories)来进行说明",
    "taste": "你自己判定的口味（口味有：sour, sweet, bitter, spicy, salty, slightly sour, slightly sweet, slightly bitter, slightly spicy, slightly salty）",
 }
]

"""

# 根据特点进行推荐
def final_recommend(user_query, model, tokenizer):
    """
    修复后的完整RAG问答函数：解决解码空白问题
    """
    print(f"🔍 正在处理查询：{user_query}")
    # 1. 解析用户需求（菜系+特征翻译）
    result = main_combine_cuisine_feature(user_query, qwen_model, qwen_tokenizer)
    # 2. RAG检索（处理空结果）
    rags_list = rag_retrieve(result, 8)
    rags_list = rags_list if rags_list else []  # 确保是列表格式

    # 3. 随机选3个（适配空列表/短列表）
    sample_count = min(3, len(rags_list))
    random_3 = random.sample(rags_list, sample_count) if sample_count > 0 else []
    random_3_str = json.dumps(random_3, ensure_ascii=False, indent=2)

    for i in range(len(random_3)):
        rag_name_lists.append(random_3[i]['meta']['Name'])
        rag_lists.append(random_3[i])

    # 4. 填充prompt
    prompt_recommend = prompt_template_recommend.replace("{{retrieved_results}}", random_3_str).replace("{{query}}", user_query)

    # 5. 模型生成（优化参数：提升稳定性）
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [
                {'role': "system", 'content': '你是专业厨师，严格按JSON格式推荐食谱，满足用户所有核心需求，理由自然详细，口味符合指定选项'},
                {"role": "user", "content": prompt_recommend}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            truncation=True,  # 开启截断（避免输入过长报错）
            max_length=4096   # 适配长RAG结果
        ).to(model.device)

        outputs = model.generate(
            inputs,
            max_new_tokens=2000,
            min_new_tokens=100,  # 确保生成足够内容
            temperature=0.7,     # 适度随机性，兼顾自然和准确
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones_like(inputs),
            repetition_penalty=1.2  # 减少重复内容
        )

    # 6. 解码+格式修复（关键：处理JSON格式错误）
    final_answer = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    """# 7. 简单格式校验（避免明显错误）
    try:
        # 尝试解析JSON，若失败返回友好提示
        json.loads(final_answer)
    except json.JSONDecodeError:
        # 修复常见格式问题（如缺少逗号、多余逗号）
        final_answer = re.sub(r',\s*]', ']', final_answer)  # 去掉列表末尾多余逗号
        final_answer = re.sub(r'\n\s*,', ',', final_answer)  # 修复换行后的多余逗号
        try:
            json.loads(final_answer)
        except:
            final_answer = '[{"Name":"out put error please try again","Calories":"","Total Time":"","Reason":"Failed to generate valid recipes","taste":""}]'"""

    print(f"✅ 推荐生成完成")
    return (final_answer, rag_lists) if final_answer.strip() else "模型已生成内容，但需调整解码逻辑（见原始内容）"

In [44]:
"""
该模块是根据菜系_特点推荐的总模块, 该模块功能包括:
    1. 调用main_combine_cuisine_feature进行菜系词、特征词的提取以及特点词的翻译以及最终的合并.
    2. 调用RAG检索，随机返回3个检索结果.
    3. 调用大模型回答用户问题.
"""
#query = 'Recommend several vegetable based delicacies'
#query = 'recommend some korea food which is suitable for some one who want to loss weight'
#query = 'recommend some chinese food that is suitable for family picnic'
#query = 'recommend some korea food which is suitable for some one who want to loss weight'
#query = 'Recommend several healthy breakfasts'
#query = 'Recommend some delicious foods suitable for fitness'
#query = 'Recommend several nutritious breakfasts suitable for children'
#query = 'Recommend several lunch options suitable for middle-aged and elderly people'
#query = "do you know any low calorie foods?"
#query = "Recommend several low protein foods suitable for health preservation"
query = 'do you know any low calorie foods?'
result,rags_list = final_recommend(query, qwen_model, qwen_tokenizer)

print(f"真正的大模型返回{result}")

🔍 正在处理查询：do you know any low calorie foods?
大模型未提取到，白名单兜底匹配
无匹配菜系：None
提取的食物特征：['fast food', 'low calorie']
最终问题TotalTime < 60 and calories < 250
✅ 推荐生成完成
真正的大模型返回 [
            {
               "Name": "fig olsons",
               "Calories": "62.8",
               "Total Time": "45.0",
               "Reason": "These Fig Olson bar cookies have relatively low calories per serving, making them suitable for weight loss goals.",
               "taste": ""
            },
            {
               "Name": "date bars",
               "Calories": "139.3",
               "Total Time": "50.0",
               "Reason": "With moderate calorie count and short preparation time, these date bars make good snacks between meals.",
               "taste": ""
            },
            {
               "Name": "food for the gods",
               "Calories": "181.8",
               "Total Time": "50.0",
               "Reason": "This Filipino dessert contains low calories but has high nutritional valu

In [45]:
import random
import json
#构建推荐提示词
prompt_template_food = """
# 立即执行：精准查询并返回食谱信息（绝对禁止多余内容）
用户查询（query）：{{query}}
你的唯一任务：
1. 输出逻辑（非A即B，无第三种可能）：
   - A. 检索结果含相关食谱（如含“kung pao chicken”）：仅输出1个JSON对象，包含且仅包含6个字段（Name、Categories、Total Time、Calories、Ingredients and Quantities、Cooking steps），字段值严格来自检索结果，无任何额外文字；
2. 字段格式铁律：
   - Ingredients and Quantities：每个食材用\\n分隔（如"a:1\\nb:2"），禁止空格/逗号连接；
   - Cooking steps：每个步骤用\\n分隔（如"1. x\\n2. y"），保留原始编号，禁止合并步骤；
   - 所有字段用双引号，无中文，无多余符号（如换行、空格）；
3. 终极禁令：
   - 绝对不允许同时输出A和B；
   - 绝对不允许输出任何解释性文字（如“The query did not return...”）；
   - 绝对不允许重复字段或步骤。

### 可用的RAG检索结果（仅基于此生成）
{{retrieved_results}}

### 输出格式（仅以下两种之一）：
{
    "Name": "{Name}",
    "Categories": "{RecipeCategory}",
    "Total Time": "{TotalTime}",
    "Calories": "{Calories}",
    "Ingredients and Quantities": "ingredient1:amount\\ningredient2:amount\\n...",
    "Cooking steps": "1. Step 1\\n2. Step 2\\n...\\nN. Step N"
}
"""

# 回答具体菜谱
def final_about_food(user_query, model, tokenizer):
    """
    修复后的完整RAG问答函数：解决解码空白问题
    """
    print(f"🔍 正在处理查询：{user_query}")
    # 2. RAG检索（处理空结果）
    rags_list = rag_retrieve(user_query, 1)
    rags_list = rags_list if rags_list else []  # 确保是列表格式
    rags_list_str = json.dumps(rags_list, ensure_ascii=False, indent=2) #转为字符串


    # 4. 填充prompt
    prompt_food = prompt_template_food.replace("{{retrieved_results}}", rags_list_str).replace("{{query}}", user_query)

    # 5. 模型生成（优化参数：提升稳定性）
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [
                {'role': "system", 'content': '你是专业厨师，严格按JSON格式推荐食谱，回答好这个美食怎么做'},
                {"role": "user", "content": prompt_food}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            truncation=True,  # 开启截断（避免输入过长报错）
            max_length=4096   # 适配长RAG结果
        ).to(model.device)

        outputs = model.generate(
            inputs,
            max_new_tokens=1000,
            min_new_tokens=20,  # 确保生成足够内容
            temperature=0.7,     # 适度随机性，兼顾自然和准确
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones_like(inputs),
            repetition_penalty=1.2  # 减少重复内容
        )

    # 6. 解码+格式修复（关键：处理JSON格式错误）
    final_answer = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    print(f"✅ 结果完成")
    return final_answer

In [46]:
query = 'do you konw how to make apple apple pie?'
result = final_about_food(query, qwen_model, qwen_tokenizer)
print(result)

🔍 正在处理查询：do you konw how to make apple apple pie?
✅ 结果完成
 {
            "Name": "Apple Pie",
            "Categories": "Pie",
            "Total Time": "80.0",
            "Calories": "479.0",
            "Ingredients and Quantities": "apples:6 -7;sugar:1;nutmeg:1;cinnamon:1;flour:2;butter:1/4;margarine:1",
            "Cooking steps": "1. Preheat oven to 350°F.\n2. Line pie pan with bottom crust.\n3. Peel and slice apples.\n4. Put 1/2 of the apple slices in prepared pie pan.\n5. Sprinkle apple slices with 1/2 cup sugar, a little nutmeg and cinnamon.\n6. Add rest of apples and top with rest of sugar (1/2 cup), a little nutmeg cinnamon, flour and butter or margarine cut into small pieces.\n7. Sprinkle with lemon juice.\n8. Cover with top crust.\n9. Crimp edges together with fork.\n10. Sprinkle top with sugar and cut steam vents in top crust.\n11. Bake for 1 hour at 350°F."
        }



In [47]:
# 清洗回答结果，防止JSON转不了
import json
import re

def parse_model_output(raw_output):
    try:
        # 第一步：直接尝试解析（正常情况）
        return json.loads(raw_output.strip())
    except json.JSONDecodeError:
        cleaned_output = raw_output.strip()
        # 1. 移除所有```及可能的json标记（包括中间的）
        cleaned_output = re.sub(r'```(json)?', '', cleaned_output)
        # 2. 修复双重闭合：只保留最后一个]或}
        if cleaned_output.count('[') > 0:
            # 处理数组类型，只保留最后一个]
            first_bracket = cleaned_output.find('[')
            last_bracket = cleaned_output.rfind(']')
            cleaned_output = cleaned_output[:first_bracket] + cleaned_output[first_bracket:last_bracket+1]
        elif cleaned_output.count('{') > 0:
            # 处理对象类型，只保留最后一个}
            first_brace = cleaned_output.find('{')
            last_brace = cleaned_output.rfind('}')
            cleaned_output = cleaned_output[:first_brace] + cleaned_output[first_brace:last_brace+1]
        # 3. 移除多余逗号（如数组最后一个元素后的逗号）
        cleaned_output = re.sub(r',\s*([}\]])', r'\1', cleaned_output)
        # 4. 修复空taste字段（补充默认值，避免解析失败）
        cleaned_output = re.sub(r'"taste":\s*""', '"taste": "unknown"', cleaned_output)
        # 再次尝试解析
        try:
            return json.loads(cleaned_output.strip())
        except json.JSONDecodeError as e:
            print(f"解析失败，原始数据：{raw_output}")
            return [
                {
                    "Name": "Data Parse Error",
                    "Calories": 0,
                    "Total Time": 0,
                    "Reason": "大模型格式输出出错，请重新输入问题吧。十分抱歉",
                    "taste": "unknown"
                }
            ]

"""
这是测试，相当于整个大模型的大脑:
 1. 第一层滤网，判定用户话题是否与美食相关
 2. 第二层滤网，判定用户话题是推荐具体食物还是推荐美食
 3. 第三层滤网，判定用户是需要根据食材来推荐，还是根据特点和特殊场景来推荐
 4. 根据判定，调用不同的函数
"""
#暴露出去的函数
def run_final(query, model, tokenizer):
    """
    暴露出去的函数，用于处理用户提问
    :param query: 用户提问
    :param model: 外部传的模型
    :param tokenizer: 外部传的分词器
    :return: 包含处理结果的字典
    """
    # 初始化返回对象（使用字典键名小写，符合Python命名规范）
    final_obj = {
        'data': None,  # 存储json或文本结果
        'rag_list': [],  # 存储RAG检索列表
        'code': 400,  # 状态码：400非食物，200具体菜，201食材推荐，202特定推荐
        'message': ""  # 新增：错误或提示信息
    }

    try:
        # 第一层滤网：判定是否为食物相关话题
        no1 = is_food_query_final_solution(query, model, tokenizer)
        if no1 != 'YES':
            print("判定为非食物话题")
            final_obj['data'] = "Please enter a topic related to food"
            final_obj['message'] = "Query is not food-related"
            return final_obj

        # 第二层滤网：判定是询问具体做法还是推荐
        no2 = whatDoYouMean(query, model, tokenizer)
        # 修复：明确判断字符串"true"/"false"（避免之前的类型转换陷阱）
        is_specific_food = no2.strip().lower() == 'true'

        if is_specific_food:
            print("判定为询问具体食物做法")
            # 修复：使用传入的model和tokenizer，而非全局变量qwen_model
            result = final_about_food(query, model, tokenizer)
            final_answer = parse_model_output(result)
            final_obj['data'] = final_answer
            final_obj['code'] = 200
            final_obj['message'] = "Success: specific food recipe"
            return final_obj
        else:
            # 第三层滤网：判定推荐类型（食材/特点）
            no3 = FinalMean(query, model, tokenizer)
            if no3.strip().lower() == 'ingredient':
                print("判定为根据食材推荐")
                # 确保函数返回2个值，避免解包错误
                result = rag_recipe_qa_fixed(query, model, tokenizer)
                if len(result) != 2:
                    raise ValueError(f"rag_recipe_qa_fixed返回值数量错误，预期2个，实际{len(result)}个")
                final_answer, rag_lists = result
                final_answer = parse_model_output(final_answer)
                final_obj['data'] = final_answer
                final_obj['rag_list'] = rag_lists
                final_obj['code'] = 201
                final_obj['message'] = "Success: ingredient-based recommendation"
                return final_obj
            else:
                print("判定为根据特点/场景推荐")
                # 同样检查返回值数量
                result = final_recommend(query, model, tokenizer)
                if len(result) != 2:
                    raise ValueError(f"final_recommend返回值数量错误，预期2个，实际{len(result)}个")
                final_answer, rag_lists = result
                final_answer = parse_model_output(final_answer)
                final_obj['data'] = final_answer
                final_obj['rag_list'] = rag_lists
                final_obj['code'] = 202
                final_obj['message'] = "Success: feature/scene-based recommendation"
                return final_obj

    except Exception as e:
        # 统一异常处理，避免程序崩溃
        print(f"处理请求时出错：{str(e)}")
        final_obj['data'] = None
        final_obj['rag_list'] = []
        final_obj['code'] = 500  # 新增：服务器内部错误码
        final_obj['message'] = f"Server error: {str(e)}"
        return final_obj


In [48]:
#最终集成测试
#query = 'how to make trump?' #判定第一层滤网，非食物
#query = 'do you know how to use iphone???' #判定第一层滤网，非食物
#query = 'do you know how to make ice cream' #第二层滤网，期望判定为制作某个具体食物
#query = 'do you know how to make kung pao chicken??' #第二层滤网，期望判定为制作某个具体食物
#query = "I have cheese, what low cal food for picnic?"  # 第三层滤网，期望判定为根据食材推荐
#query = 'I have eggs, chocolate, and milk. what kind of food can i make?'  # 第三层滤网，期望判定为根据食材推荐
#query = "do you know any low calorie foods?" # 第三层滤网，期望判定为根据菜系特点来回答
#query = "food for a cave exploration trip"  # 第三层滤网，期望判定为根据菜系特点来回答
#query = 'give me some spicy chinese food' # 第三层滤网，判定为根据菜系特点来回答
#query = 'Recommend several foods suitable for weight loss' #第三层滤网，判定为根据菜系特点来回答

final_obj = run_final(query,qwen_model,qwen_tokenizer)
print(f"final_answer: {final_obj}")

🔍 模型核心判定：[do you konw how to make apple apple pie?] → 输出：[YES]
判定为询问具体食物做法
🔍 正在处理查询：do you konw how to make apple apple pie?
✅ 结果完成
final_answer: {'data': {'Name': 'Apple Pie', 'Categories': 'Pie', 'Total Time': '80.0', 'Calories': '479.0', 'Ingredients and Quantities': 'apples:6\nsugar:1\nnutmeg:1\ncinnamon:1\nflour:2\nbutter:1/4\nmargarine:1', 'Cooking steps': '1. Preheat oven to 350°F.\n2. Line pie pan with bottom crust.\n3. Peel and slice apples.\n4. Put 1/2 of the apple slices in prepared pie pan.\n5. Sprinkle apple slices with 1/2 cup sugar, a little nutmeg and cinnamon.\n6. Add rest of apples and top with rest of sugar (1/2 cup), a little nutmeg cinnamon, flour and butter or margarine cut into small pieces.\n7. Sprinkle with lemon juice.\n8. Cover with top crust.\n9. Crimp edges together with fork.\n10. Sprinkle top with sugar and cut steam vents in top crust.\n11. Bake for 1 hour at 350°F.'}, 'rag_list': [], 'code': 200, 'message': 'Success: specific food recipe'}
